# Razvejanost kraljestva živali


<img src="zivali.jpg"></img> 

Na predavanjih smo spoznali algoritem <b>hieraričnega gručenja</b>. Njegova glavna značilnost je, da omogoča primerjavo objektov zgolj na podlagi poznavanja <i> mere razdalje </i> med njimi. Predstavitev podatkov torej ni nujno omejena na vektorske prostore.

Algoritem je determinističen in ne predpostavlja števila gruč. Rezultat gručenja bo izračunan naenkrat za vsa možna števila gruč v intervalu $[1, n]$, odločitev o številu pa bo sprejeta po izračunu.



<font color="blue"><b>Razmisli.</b></font> Kakšna je časovna zahtevnost algoritma za hierarhično gručenje? Kako se primerja z metodo K-means?

In [42]:
import numpy as np

% matplotlib inline
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import scipy

Današnji podatki spominjajo na (starejši generaciji dobro znani) album sličic z živalmi. Vsebuje 59 živalskih vrst ter 16 atributov, ki opisuje pripadajoče anatomske značilnosti. Živali so razdeljene v 7 razredov.

In [43]:
data = np.genfromtxt("zoo.tab", skip_header=3)[::, -2:-1]

print(data[:10])

[[ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]]


# Zanimala nas bo predvsem matrika `X`, ki podatke hrani v številski obliki.

In [44]:
X =  np.genfromtxt("zoo.tab", skip_header=3)[::, :-2]
print(X.shape)

(59, 16)


Rezultat gručenja dobimo z uporabo modula `scipy.cluster.hierarchy` in metode `linkage`. Slednja izračune povezave v drevesu (dendrogramu) glede na dano mero razdalje (`metric`) in načinom merjenja razdalj med gručami (`method`).

In [45]:
L = sch.linkage(X, method="single", metric="euclidean")   

Z uporabo funkcije `dendrogram` narišemo drevo in mu priredimo oznake. Funkcija deluje v navezi z že znano knjižnico `matplotlib`.

In [47]:
plt.figure(figsize=(25, 6))
labels = [row["name"].value for row in data]
D      = sch.dendrogram(L, labels=labels, p=20, leaf_font_size=15)
plt.ylabel("Razdalja")
plt.show()

/home/amon/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

V redu, za prvi poizkus. Vseeno izgleda dendrogram nekoliko sploščen. Preveri, kako na graf vplivajo različne ...

### ... metode povezovanja

Metode povezovanja določajo način, kako izračunati razdaljo med dvema poljubno velikima gručama točk.
* <b>Posamično povezovanje</b> (method="single"); Razdalja med gručama je razdalja med najbližjima točkama gruč.
* <b>Povprečna razdalja</b> (method="average"); Povprečna razdalja med vsemi pari točk. 
* <b>Razdalja med središčema</b> (method="centroid"); Izračuna središči gruč v prostoru ter njuno medsebojno razdaljo. Mera razdalje je nujno evklidska.

<font color="green"><b>Naredi sam/a.</b></font> Preizkusi različne oblike dendrograma glede na izbrano mero razdalje.

In [48]:
# Preizkusi različne načine merjenja razdalje med gručami
L = sch.linkage(X, method="single",)

Ali je evklidska razdalja res najbolj primeren način primerjanja atributov, ki so diskretni? Ne vedno.

### Mere razdalje

Način določanja interpretacije razdalje med točkama $\vec{x} = (x_1, x_2, ..., x_p)$ in $\vec{y} = (y_1, y_2, ..., y_p)$ vpliva na rezultat hierarhičnega gručenja. Izbira ustrezne mere je odvisna od narave podatkov in čimbolje odgovarja na vprašanje: <i>kaj pomeni, da sta dva primera podobna?</i>

Na izbiro ustrezne mere lahko vplivajo:
* Prisotnost manjkajočih vrednosti
* Predstavitev podatkov (vektorji, nizi znakov, slike, ...)
* Tip atributov in interpretacija vrednosti


Nekaj pogostih mer razdalje:
* Evklidska razdalja <code>(metric="euclidean")</code>

$$ d(\vec{x}, \vec{y}) = \sqrt{(x_1-y_1)^2 + (x_2 - y_2)^2 + ... + (x_p - y_p)^2} $$

* Manhattanska razdalja <code>(metric="cityblock")</code>

$$ d(\vec{x}, \vec{y}) = |x_1-y_1| + |x_2 - y_2| + ... + |x_p - y_p| $$

* Kosinusna razdalja <code>(metric="cosine")</code>

<p>Predstavlja <i>kosinus kota</i> med vektorjema $\vec{x}$ in $\vec{y}$ - manjši kot pomeni večjo podobnost. Uporabna za primerjavo podobnosti med vektorji, neupoštevajoč absolutnih velikosti.</p>

$$ d(\vec{x}, \vec{y}) = 1 - \frac{\vec{x} \cdot \vec{y}}{\|\vec{x}\| \|\vec{y}\|} $$

* Jaccardov index <code>(metric="jaccard")</code>

<p>Izmeri delež ujemanj med pari soležnih komponent $(x_i, y_i)$, kjer je vsaj ena izmed vrednosti $x_i$ ali $y_i$ večja od nič. Primerna za uporabo v primerih, ko imamo opravka z manjkajočimi vrednostmi ali diskretnimi atributi.</p>

Popoln spisek razdalj najdeš v <a href="http://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.spatial.distance.pdist.html">dokumentaciji</a>.

<br/>
<font color="blue"><b>Razmisli.</b></font> Poizkusi se spomniti vrste podatkov, kjer bi bilo smiselno uporabiti vsako posamezno mero.

### Določanje števila gruč

Koliko gruč je v podatkih? Na to vprašanje je težko odgovoriti in tudi sicer velja za odprto vprašanje na področju storjnega učenja. Vseeno poznamo nekaj kazalcev, ki jih v grobem delimo na 
* <font color="green">nadzorovane</font> (znani so resnični razredi podatkov)
* <font color="blue">nenadzorovane</font> (znane so samo značilke in/ali razdalje med primeri)

Za določitev pripadnosti primerov gručam uporabimo funkcijo `fcluster`. Slednja prejme parameter `t`, ki določa razdaljo pri kateri <i>odsekamo</i> dendrogram, t.j. odstranimo vse povezave, ki so daljše od dane dolžine. Preostale povezane komponente grafa dendrograma tako tvorijo skupine.

In [49]:
t = 3
predictions = sch.fcluster(L, t=t, criterion="distance").ravel()
classes     = data   # resnicni razredi 

print("Primer", "Resnični razred", "Gruča")
for row, category, prediction in list(zip(data, predictions, classes ))[2:10]:
        print("%s\t%d\t%d" % (row["name"], category, prediction))

Primer Resnični razred Gruča


/home/amon/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

Ponovno narišemo dendrogram in ga <i>odsekamo</i> pri dani razdalji. V nadaljevanju si bomo ogledali številske ocene uspešnosti gručenja.

In [26]:
D = sch.dendrogram(L, labels=labels)
plt.plot([0, 1000], [t, t], "k--")
plt.ylabel("Razdalje")
plt.show()

NameError: name 'labels' is not defined

#### <font color="green">Skupna deljena informacija</font>

Mera skupne deljene informacije je uporabna, ko so na voljo informacije o resničnih razredih, v katere spadajo primeri. Pri tem ni odveč poudariti, da resnični razredi ne smejo biti uporabljeni pri deljenju primerov v skupine.

Naključne dodelitve oznak gruč imajo vrednost skupne deljene informacije 
blizu `0.0` za vsako vrednost števila skupin in števila primerov. Popolno ujemanje gruč z obstoječimi razredi ima vrednost `1`.
Mera ni odvisna od predstavitve podatkov, t.j. ni potrebno da so podatki v vektorskem prostoru, saj je odvisna samo od oznak.

In [28]:
from sklearn.metrics import adjusted_mutual_info_score
score = adjusted_mutual_info_score(classes, predictions)
score

ValueError: labels_true must be 1D: shape is (59, 1)

#### <font color="blue">Koeficitent silhuete</font>

Koeficient silhuete je nenadzorovana meta v območju med `-1` (napačno dodeljene skupine) in `1` (zelo goste, dobro ločene skupine). Večja notranja gostota znotraj skupin in večja razdalja sta premosorazmeni s koeficientom. Tudi ta mera ne predpostavlja, da so podatki v vektorskem prostoru, je pa odvisna od izbrane mere razdalje.

<font color="green"><b>Naredi sam/a.</b></font> Preveri, kako se ocena spreminja glede na izbrano mero razdalje. Katera mera razdalje najbolje oceni gručenje? Ali je rezultat smiseln?

In [41]:
from sklearn.metrics import silhouette_score

score = silhouette_score(X, predictions, metric="euclidean")
score

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

<font color="green"><b>Naredi sam/a.</b></font> Izvedi analizo gručenja na podatkih o živalih  tako, da izbereš ustrezno metodo povezovanja, mero razdalje in število gruč. Uporabi eno od predstavljenih mer podobnosti ter poišči tako kombinacijo omenjenih nastavitev, da bo rezultat gručenja karseda visok.

In [30]:
# ...

## Podatki v obliki nizov znakov

Stopnja razvoja na področju biotehnologije omogoča pridobivanje bistveno več podatkov o organizmih. Eden pogostih podatkovnih tipov, s katerimi primerjamo vrste so genske zapisi. Ti so pripravni za predstavitev v računalništvu, saj jih lahko posplošimo na zaporedna štirih nukleotidov: A, C, G, T. Celoten genski zapis ki določa vse, od vaše barve oči do nagnjenosti do določenih bolezni je podano z nekaj več kot $3 \times 10^{12}$ dolgim zaporednjem DNK.

Pri razmoževanju prihaja do prepisovanja in kombiniranja DNA zapisov staršev. Ta proces seveda ni popoln, zato prihaja do napak - <i>mutacij</i>. Dolgoročna posledica mutacij pa je natanek različnih živalskih vrst, kar pomeni, da imajo sorodnejše vrste bolj podobne genske zapise.

Iz baze genskih zapisov smo naložili zaporednja mitohondrijskega gena za 13 vrst: `'Gorilla gorilla', 'Homo sapiens', 'Carassius auratus auratus', 'Delphinus capensis', 'Chamaeleo calyptratus', 'Canis lupus familiaris', 'Homo sapiens neanderthalensis', 'Rattus norvegicus', 'Equus caballus', 'Daboia russellii', 'Pan troglodytes', 'Takifugu rubripes', 'Pongo abelii', 'Sus scrofa'`.

In [295]:
import json
sequences = json.load(open("seqs.json"))
print(sequences["Homo sapiens"][:100]) 
print(sequences["Gorilla gorilla"][:100]) 
print(len(sequences["Homo sapiens"]))

GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCATGCATTTGGTATTTTCGTCTGGGGGGTATGCACGCGATAGCATTGCGAGACGCTG
GTTTATGTAGCTTACCTCCCCAAAGCAATACACTGAAAATGTTTCGACGGGCTCACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAA
16569


<font color="green"><b>Naredi sam/a.</b></font> Kako bi lahko primerjali živalske vrste glede na zapise, ki so podani kot nizi znakov? Prva ideja je, da podatke pretvorimo v vektorski prostor, v katerem bomo računali razdalje. Namig: zaporedja lahko razbiješ na manjše dele in prešteješ število pojavitev posameznih znakov, parov, trojk, ... k-terk. ahko upoštevaš tudi položaj v zaporedju. 

Dopolni in si pomagaj s funkcijo `seq_to_kmer_count`, ki pretvori niz znako v vektor števila pojavitev vseh mogožnih k-terk.

Prevedi podatke v ustrezno obliko, izvedi hierarhično gručenje in prikaži rezultate. Ali so vrste na dendrogramu postavljene smiselno? 

In [293]:
from itertools import product
def seq_to_kmer_count(seq, k=4):
    """
    Pretvori zaporedje seq v vektor x.
         AAAA AAAC AAAG AAAT ... TTTG TTTT
    x = [   1  1      2   10 ...   12    7]
    """
    
    ktuples = list(zip(*[seq[i:] for i in range(k)]))     # razbijemo trenutni niz seq na k-terke
    kmers   = list(product(*(k*[["A", "C", "T", "G"]])))  # vse mozne k-terke
    
    x = np.zeros((len(kmers), ))
    ### Your code here ### 
    
    return x